In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
#pip list
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -qq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 7

In [5]:
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/malayalam_dataset_2_train.csv")
dataset2_filtered = dataset2[dataset2['label'] != 2]
X=list(dataset2_filtered['text'])
y=list(dataset2_filtered['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=42,test_size=0.3)
train_df = pd.DataFrame({"text": X_train, "labels": y_train})
test_df = pd.DataFrame({"text": X_test, "labels": y_test})

In [6]:
train_df.shape

(12882, 2)

In [7]:
test_df.shape

(5521, 2)

In [8]:
import torch
torch.cuda.device_count()

1

In [9]:
device = torch.cuda.get_device_properties(0)  # Assuming you have at least one GPU
print(device)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [11]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [10]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [12]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

# Malayalam_BERT

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 17,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 2,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'learning_rate': 2e-5,
    'train_batch_size': 64,
    "gradient_accumulation_steps":3,
    "use_early_stopping": True,
}

mabert_model_malayalam = ClassificationModel(
    "bert",
    "l3cube-pune/malayalam-bert",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/malayalam-bert and are newly initialized: ['classifier.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
mabert_model_malayalam.train_model(train_df)

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch:   0%|          | 0/17 [00:00<?, ?it/s]

Running Epoch 1 of 17:   0%|          | 0/202 [00:00<?, ?it/s]

Running Epoch 2 of 17:   0%|          | 0/202 [00:00<?, ?it/s]

Running Epoch 3 of 17:   0%|          | 0/202 [00:00<?, ?it/s]

...

Running Epoch 14 of 17:   0%|          | 0/202 [00:00<?, ?it/s]

Running Epoch 15 of 17:   0%|          | 0/202 [00:00<?, ?it/s]

Running Epoch 16 of 17:   0%|          | 0/202 [00:00<?, ?it/s]

Running Epoch 17 of 17:   0%|          | 0/202 [00:00<?, ?it/s]

(1717, 0.1862157629455679)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = mabert_model_malayalam.eval_model(test_df)

  0%|          | 0/11 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
print(result) #result of the model

{'mcc': 0.0, 'accuracy': 0.9587031334903098, 'f1_score': 0.0, 'tp': 0, 'tn': 5293, 'fp': 0, 'fn': 228, 'auroc': 0.3857635539822539, 'auprc': 0.035252823737401004, 'eval_loss': 0.1771674059065325}


In [ ]:
predictions, raw_outputs = mabert_model_malayalam.predict(list(test_df['text'].values))

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['labels'].values,zero_division=True))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98      5521
           1       0.00      1.00      0.00         0

    accuracy                           0.96      5521
   macro avg       0.50      0.98      0.49      5521
weighted avg       1.00      0.96      0.98      5521



In [ ]:
import pickle
# save the model to disk
filename = 'mabert_malayalam.sav'
pickle.dump(mabert_model_malayalam, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
accuracy = accuracy_score(test_df['labels'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9587031334903098


In [ ]:
f1 = f1_score(test_df['labels'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.9384900472654355


In [ ]:
import shutil

local_model_path = '/content/sample_data/mabert_malayalam.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Malayalam/mabert_malayalam.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Malayalam/mabert_malayalam.sav'

In [13]:
import pickle
loaded_model=pickle.load(open("/content/drive/MyDrive/Capstone/Models/Malayalam/mabert_malayalam.sav",'rb'))

In [17]:
print(loaded_model.predict(["ഇതു വരെ 28 k ലൈക്‌ വന്നു lady super star മഞ്ജു ചേച്ചിക്ക് ... 1.1 k കോവാലൻ പാൻസ്‌ dislike അടിച്ചു"])[0])

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]


In [ ]:
#CATBOOST